# Parte 1: Implementación práctica

Marco Jurado 20308

In [1]:
#!pip install lightgbm
#!pip install --upgrade lightgbm
#!pip install category_encoders
#!pip install geopy

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from geopy.distance import geodesic
import numpy as np
import logging

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
data = pd.read_csv('fraud_feature_engineering_example.csv')
data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,4.97,4.97,0.00,0.0,True
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,107.23,107.23,0.00,0.0,True
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,220.11,220.11,0.00,0.0,True
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,45.00,45.00,0.00,0.0,True
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,41.96,41.96,0.00,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0,11619.63,72134.23,1014.44,11.0,False
1852390,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0,15224.47,87115.43,3942.78,25.0,False
1852391,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0,26233.12,165389.30,2978.91,29.0,False
1852392,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0,11787.71,90698.65,768.69,17.0,False


In [5]:
data.shape

(1852394, 27)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 27 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   trans_date_trans_time         object 
 1   cc_num                        int64  
 2   merchant                      object 
 3   category                      object 
 4   amt                           float64
 5   first                         object 
 6   last                          object 
 7   gender                        object 
 8   street                        object 
 9   city                          object 
 10  state                         object 
 11  zip                           int64  
 12  lat                           float64
 13  long                          float64
 14  city_pop                      int64  
 15  job                           object 
 16  dob                           object 
 17  trans_num                     object 
 18  unix_time             

Eliminaremos algunas columnas irrelevantes

In [7]:
columns_to_drop = ['first', 'last', 'city', 'state', 'zip', 'city_pop', 'job', 'dob']
data.drop(columns_to_drop, axis=1, inplace=True)

In [8]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,lat,long,trans_num,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,561 Perry Cove,36.0788,-81.1781,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,4.97,4.97,0.0,0.0,True
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,107.23,107.23,0.0,0.0,True
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,594 White Dale Suite 530,42.1808,-112.2620,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,220.11,220.11,0.0,0.0,True
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,9443 Cynthia Court Apt. 038,46.2306,-112.1138,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,45.00,45.00,0.0,0.0,True
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,408 Bradley Rest,38.4207,-79.4629,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,41.96,41.96,0.0,0.0,True


In [9]:
data.describe()

,cc_num,amt,lat,long,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net
count,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06
mean,4.173860e+17,7.006357e+01,3.853931e+01,-9.022783e+01,1.358674e+09,3.853898e+01,-9.022794e+01,5.210015e-03,4.153689e+03,4.530560e+04,3.762028e+02,4.567241e+00
std,1.309115e+18,1.592540e+02,5.071470e+00,1.374789e+01,1.819508e+07,5.105604e+00,1.375969e+01,7.199217e-02,3.909005e+03,3.586752e+04,7.253531e+02,4.575502e+00
min,6.041621e+10,1.000000e+00,2.002710e+01,-1.656723e+02,1.325376e+09,1.902742e+01,-1.666716e+02,0.000000e+00,1.000000e+00,1.020000e+00,0.000000e+00,0.000000e+00
25%,1.800429e+14,9.640000e+00,3.466890e+01,-9.679800e+01,1.343017e+09,3.474012e+01,-9.689944e+01,0.000000e+00,1.344790e+03,1.734142e+04,9.020000e+00,1.000000e+00
50%,3.521417e+15,4.745000e+01,3.935430e+01,-8.747690e+01,1.357089e+09,3.936890e+01,-8.744069e+01,0.000000e+00,3.071990e+03,3.743910e+04,7.589000e+01,3.000000e+00
75%,4.642255e+15,8.310000e+01,4.194040e+01,-8.015800e+01,1.374581e+09,4.195626e+01,-8.024511e+01,0.000000e+00,5.738470e+03,6.472088e+04,4.259800e+02,7.000000e+00
max,4.992346e+18,2.894890e+04,6.669330e+01,-6.795030e+01,1.388534e+09,6.751027e+01,-6.695090e+01,1.000000e+00,4.326189e+04,2.190868e+05,1.204718e+04,4.800000e+01


In [10]:
data["merchant"].value_counts()

fraud_Kilback LLC                        6262
fraud_Cormier LLC                        5246
fraud_Schumm PLC                         5195
fraud_Kuhn LLC                           5031
fraud_Boyer PLC                          4999
                                         ... 
fraud_Douglas, DuBuque and McKenzie      1101
fraud_Treutel-King                       1098
fraud_Satterfield-Lowe                   1095
fraud_Hahn, Douglas and Schowalter       1091
fraud_Ritchie, Bradtke and Stiedemann    1090
Name: merchant, Length: 693, dtype: int64

In [11]:
# proporcion 1 a 5 es aceptable (OJO)

data["is_fraud"].value_counts()

0    1842743
1       9651
Name: is_fraud, dtype: int64

In [12]:
data["cc_num"].value_counts()

6538441737335434       4392
30270432095985         4392
6538891242532018       4386
4364010865167176       4386
4642255475285942       4386
                       ... 
4714017207228610634       7
4352307151555405069       7
180097223252063           7
6011732631674654          7
4295296907373             6
Name: cc_num, Length: 999, dtype: int64

In [13]:
data["trans_date_trans_time"] = pd.to_datetime(data["trans_date_trans_time"])

In [14]:
data = data.sort_values(by='trans_date_trans_time')

In [15]:
data["cc_num"] = data["cc_num"].astype(str)

In [16]:
data["trans_num"].value_counts(dropna=False)

0b242abb623afc578575680df30655b9    1
ad44105b2defec28687e0c0c00db0dd2    1
fff81009b323191d0a427a2af21e5bc7    1
2341854d7594722011878c08ba3819dd    1
f7f00d638b16a8b1045d8a8978b607bd    1
                                   ..
74745c7e44a5172e919b2ea2538d0edb    1
71d44820f944faa2ed912043e859df48    1
986d5219c856c1ddcafe1a4ad35b844e    1
cf4fca8a9847c75240202201324dd34f    1
1765bb45b3aa3224b4cdcb6e7a96cee3    1
Name: trans_num, Length: 1852394, dtype: int64

In [17]:
data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'gender', 'street', 'lat', 'long', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'amt_month', 'amt_year',
       'amt_month_shopping_net_spend', 'count_month_shopping_net',
       'first_time_at_merchant'],
      dtype='object')

## Generar nuevas caracteristicas

#### Tiempo desde la última transacción

In [18]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
data['time_since_last_txn'] = data.groupby('cc_num')['trans_date_trans_time'].diff().apply(lambda x: x.total_seconds())

#### Cantidad promedio de transacciones por día

In [19]:
data['date'] = data['trans_date_trans_time'].dt.date
daily_amount = data.groupby(['cc_num', 'date'])['amt'].sum().groupby('cc_num').mean().reset_index()
daily_amount.columns = ['cc_num', 'avg_daily_amt']
data = pd.merge(data, daily_amount, on='cc_num', how='left')

#### Número de transacciones en la última hora

In [20]:
data['hour'] = data['trans_date_trans_time'].dt.hour
transactions_last_hour = data.groupby(['cc_num', 'date', 'hour'])['trans_num'].count().groupby('cc_num').mean().reset_index()
transactions_last_hour.columns = ['cc_num', 'avg_hourly_transactions']
data = pd.merge(data, transactions_last_hour, on='cc_num', how='left')


#### Distancia desde la última transacción

In [21]:
def calculate_distance(row):
    if pd.isna(row['prev_lat']) or pd.isna(row['prev_long']) or pd.isna(row['merch_lat']) or pd.isna(row['merch_long']):
        return 0.00

    coords_1 = (row['prev_lat'], row['prev_long'])
    coords_2 = (row['merch_lat'], row['merch_long'])
    try:
        distance = geodesic(coords_1, coords_2).km
        return distance
    except ValueError as e:
        logging.error(f"Error calculating distance for row: {e}")
        return np.nan

In [22]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
data['prev_lat'] = data.groupby('cc_num')['lat'].shift(1)
data['prev_long'] = data.groupby('cc_num')['long'].shift(1)


In [23]:
data['distance_from_last_txn'] = data.apply(calculate_distance, axis=1)

In [24]:
print(data[['cc_num', 'trans_date_trans_time', 'distance_from_last_txn']].head())

            cc_num trans_date_trans_time  distance_from_last_txn
0  180011453250192   2019-01-03 01:39:04                0.000000
1  180011453250192   2019-01-03 05:51:22               47.805838
2  180011453250192   2019-01-03 14:48:47               69.701338
3  180011453250192   2019-01-04 10:39:10               58.825873
4  180011453250192   2019-01-04 12:48:21               83.476707


#### Variación de gastos mensuales

In [25]:
data['month_year'] = data['trans_date_trans_time'].dt.to_period("M")

In [26]:
monthly_spending = data.groupby(['cc_num', 'month_year'])['amt'].sum()
monthly_spending_variation = monthly_spending.groupby(level=0).pct_change().reset_index()
monthly_spending_variation.columns = ['cc_num', 'month_year', 'monthly_spending_variation']

In [27]:
data = pd.merge(data, monthly_spending_variation, on=['cc_num', 'month_year'], how='left')

#### Día de la transacción y hora de la transacción

In [28]:
data['weekday'] = data['trans_date_trans_time'].dt.dayofweek
data['hour'] = data['trans_date_trans_time'].dt.hour

#### Días festivos para detección de posibles patrones

In [29]:
import holidays
us_holidays = holidays.US()
data['is_holiday'] = data['trans_date_trans_time'].dt.date.apply(lambda x: x in us_holidays)

#### Tiempo desde la primera transacción

In [30]:
data['first_txn_time'] = data.groupby('cc_num')['trans_date_trans_time'].transform('min')
data['days_since_first_txn'] = (data['trans_date_trans_time'] - data['first_txn_time']).dt.days

#### Cantidad de transacciones en los últimos X minutos/horas

In [31]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
data.set_index('trans_date_trans_time', inplace=True)
data['txn_count_last_24h'] = data.groupby('cc_num')['trans_num'].rolling('24h').count().values
data.reset_index(inplace=True)


#### Cantidad promedio y desviación estándar del monto en un periodo reciente

In [32]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
data.set_index('trans_date_trans_time', inplace=True)
data['avg_amt_last_24h'] = data.groupby('cc_num')['amt'].transform(
    lambda x: x.rolling('24h').mean()
)
data['std_amt_last_24h'] = data.groupby('cc_num')['amt'].transform(
    lambda x: x.rolling('24h').std()
)
data.reset_index(inplace=True)

#### Ratio de transacciones internacionales o en línea

In [33]:
data['online_txn_ratio'] = data.groupby('cc_num')['category'].apply(lambda x: (x == 'shopping_net').mean())

#### Transacciones Mensuales por Tarjeta y Categoría

In [34]:
data['trans_date'] = data['trans_date_trans_time'].dt.to_period('M')
monthly_count = data.groupby(['cc_num', 'category', 'trans_date']).cumcount() + 1
data['monthly_txn_count_by_cat'] = monthly_count

#### Transacciones Anuales por Tarjeta y Categoría

In [35]:
data['trans_year'] = data['trans_date_trans_time'].dt.year
yearly_count = data.groupby(['cc_num', 'category', 'trans_year']).cumcount() + 1
data['yearly_txn_count_by_cat'] = yearly_count

#### Transacciones por Temporada por Tarjeta y Categoría

In [36]:
def map_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

data['season'] = data['trans_date_trans_time'].dt.month.apply(map_season)

In [37]:
seasonal_count = data.groupby(['cc_num', 'category', 'season']).cumcount() + 1
data['seasonal_txn_count_by_cat'] = seasonal_count

#### Velocidad de Transacción

In [38]:
data['time_diff_hours'] = data.groupby('cc_num')['trans_date_trans_time'].diff().astype('timedelta64[h]')
data['transaction_speed'] = data['distance_from_last_txn'] / data['time_diff_hours']
data['transaction_speed'].fillna(0, inplace=True)

#### Ratios de Gasto

In [39]:
data['amt_to_daily_avg'] = data['amt'] / data['avg_daily_amt']
monthly_avg_amt = data.groupby(['cc_num', 'month_year'])['amt'].transform('mean')
data['amt_to_monthly_avg'] = data['amt'] / monthly_avg_amt

#### Frecuencia de Transacciones en Cortos Períodos

In [40]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.set_index('trans_date_trans_time', inplace=True)
data['txn_count_last_hour'] = data.groupby('cc_num')['trans_num'].rolling('1h').count().values
data.reset_index(inplace=True)

#### Uso de Nuevos Comerciantes

In [41]:
data.sort_values(by=['cc_num', 'trans_date_trans_time'], inplace=True)
data['new_merchant'] = (~data['merchant'].duplicated()) & (data['merchant'] != data.groupby('cc_num')['merchant'].shift(1))

#### Cambio de Patrón de Gasto

In [42]:
if not isinstance(data.index, pd.DatetimeIndex):
    if 'trans_date_trans_time' in data.columns:
        data.set_index('trans_date_trans_time', inplace=True)
    else:
        data.reset_index(inplace=True)
        data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
        data.set_index('trans_date_trans_time', inplace=True)
data['year'] = data.index.year
annual_avg_amt = data.groupby(['cc_num', 'year'])['amt'].transform('mean')
data['amt_to_annual_avg'] = data['amt'] / annual_avg_amt


In [43]:
num_columns = len(data.columns)
print("Número de columnas:", num_columns)

Número de columnas: 50


In [44]:
data.columns

Index(['cc_num', 'merchant', 'category', 'amt', 'gender', 'street', 'lat',
       'long', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud',
       'amt_month', 'amt_year', 'amt_month_shopping_net_spend',
       'count_month_shopping_net', 'first_time_at_merchant',
       'time_since_last_txn', 'date', 'avg_daily_amt', 'hour',
       'avg_hourly_transactions', 'prev_lat', 'prev_long',
       'distance_from_last_txn', 'month_year', 'monthly_spending_variation',
       'weekday', 'is_holiday', 'first_txn_time', 'days_since_first_txn',
       'txn_count_last_24h', 'avg_amt_last_24h', 'std_amt_last_24h',
       'online_txn_ratio', 'trans_date', 'monthly_txn_count_by_cat',
       'trans_year', 'yearly_txn_count_by_cat', 'season',
       'seasonal_txn_count_by_cat', 'time_diff_hours', 'transaction_speed',
       'amt_to_daily_avg', 'amt_to_monthly_avg', 'txn_count_last_hour',
       'new_merchant', 'year', 'amt_to_annual_avg'],
      dtype='object')

Finalmente limpiamos las ultimas variables que pueden generar que nuestros modelos tengan tendencias que no queremos

In [45]:
existing_columns_to_drop = [col for col in ['gender', 'street', 'date', 'year'] if col in data.columns]
if existing_columns_to_drop:
    data.drop(columns=existing_columns_to_drop, axis=1, inplace=True)

In [46]:
data.reset_index(inplace=True)

In [47]:
data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'lat',
       'long', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud',
       'amt_month', 'amt_year', 'amt_month_shopping_net_spend',
       'count_month_shopping_net', 'first_time_at_merchant',
       'time_since_last_txn', 'avg_daily_amt', 'hour',
       'avg_hourly_transactions', 'prev_lat', 'prev_long',
       'distance_from_last_txn', 'month_year', 'monthly_spending_variation',
       'weekday', 'is_holiday', 'first_txn_time', 'days_since_first_txn',
       'txn_count_last_24h', 'avg_amt_last_24h', 'std_amt_last_24h',
       'online_txn_ratio', 'trans_date', 'monthly_txn_count_by_cat',
       'trans_year', 'yearly_txn_count_by_cat', 'season',
       'seasonal_txn_count_by_cat', 'time_diff_hours', 'transaction_speed',
       'amt_to_daily_avg', 'amt_to_monthly_avg', 'txn_count_last_hour',
       'new_merchant', 'amt_to_annual_avg'],
      dtype='object')

In [48]:
try:
    data.to_csv('final_dataset.csv', index=False)
    print("Dataset guardado como 'final_dataset.csv'.")
except Exception as e:
    print(f"Ocurrió un error al guardar el archivo: {e}")


Dataset guardado como 'final_dataset.csv'.
